In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import libraries
import numpy as np
import sys
import psi4
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json
import matplotlib.pyplot as plt
import json
import os

# import libraries
import numpy as np
import sys
import psi4
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json
import matplotlib.pyplot as plt
import json
import os 


$$ |\psi_n^{(k)}\rangle = \sum_{m \neq n} \frac{\langle \psi_m^{(0)} | \hat{H}' | \psi_n^{(k-1)} \rangle}{E_n^{(0)} - E_m^{(0)}} |\psi_m^{(0)} \rangle $$
$$ E_n^{(k)} = \langle \psi_n^{(0)} | \hat{H}' | \psi_n^{(k-1)} \rangle $$

Note:

$$ \hat{U}_{CS} \hat{b} \hat{U}^{\dagger}_{CS} = \hat{b} - z $$

$$ \hat{U}_{CS} \hat{b}^{\dagger} \hat{U}^{\dagger}_{CS} = \hat{b}^{\dagger} - z $$

where 

$$ z = -\frac{\langle \hat{d} \rangle}{\sqrt{2 \omega}} $$


In [ ]:



class PT_arbitrary:

    def __init__(self, energies, dipoles):
        self.E_array = energies
        self.dipoles = dipoles




    def create_annihilation_operator(self,N):
        """
        Creates the matrix representation of the annihilation operator (b) for a harmonic oscillator
        in a Hilbert space with N levels.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the annihilation operator.
        """
        b = np.zeros((N, N))
        
        for j in range(1, N):
            b[j-1, j] = np.sqrt(j)
        
        return b

    def create_cs_annihilation_operator(self,N, z):
        """
        Creates the matrix representation of the annihilation operator (b) for a harmonic oscillator
        in a Hilbert space with N levels.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the annihilation operator.
        """
        b = np.zeros((N, N))

        _I = np.eye(N)
        
        for j in range(1, N):
            b[j-1, j] = np.sqrt(j)

        b -= z * _I
        return b
        
    def create_creation_operator(self,N):
        """
        Creates the matrix representation of the creation operator (b†) for a harmonic oscillator
        in a Hilbert space with N levels.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the creation operator.
        """
        b_dagger = np.zeros((N, N))
        
        for j in range(1, N):
            b_dagger[j, j-1] = np.sqrt(j)
        
        return b_dagger

    def create_cs_creation_operator(self,N, z):
        """
        Creates the matrix representation of the creation operator (b†) for a harmonic oscillator
        in a Hilbert space with N levels.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the creation operator.
        """
        b_dagger = np.zeros((N, N))

        _I = np.eye(N)
        
        for j in range(1, N):
            b_dagger[j, j-1] = np.sqrt(j)

        b_dagger -= z * _I
        return b_dagger


    def create_number_operator(self,N):
        """
        Creates the matrix representation of the number operator (n) = b† * b.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the number operator.
        """
        b = self.create_annihilation_operator(N)
        b_dagger = self.create_creation_operator(N)
        
        # The number operator is n = b† * b
        n = np.dot(b_dagger, b)
        
        return n

    def create_cs_number_operator(self,N, z):
        """
        Creates the matrix representation of the number operator (n) = b† * b.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the number operator.
        """
        b = self.create_cs_annihilation_operator(N, z)
        b_dagger = self.create_cs_creation_operator(N, z)
        
        # The number operator is n = b† * b
        n = np.dot(b_dagger, b)
        
        return n


    def build_d_array(
        self,
        n_el,
        lambda_vector,
        mu_array
        ):
        """
        method to compute the array d = \lambda \cdot \mu if coherent_state==False
        or d = \lambda \cdot (\mu - <\mu>) if coherent_state == True
        and store to attribute self.d_array
        """

        d_array = np.einsum("k,ijk->ij", lambda_vector, mu_array[:n_el, :n_el, :])


        return d_array

    def compute_energy_corrections(self, order, n_el, n_ph, omega, lambda_vector,  coherent_state = False):

        n_max = n_el * n_ph
        n = 0

        # Identity matrices for each subsystem
        I_matter = np.eye(n_el)
        I_photon = np.eye(n_ph)

        # Build dipole moment matrix _d
        d_array = self.build_d_array(n_el, lambda_vector, self.dipoles)
        _d = np.copy(d_array)

        # assign z for the coherent state transformation
        _z = - _d[0,0] / np.sqrt(2 * omega)


        # Create bosonic subspace operators
        if coherent_state == False:
            b = self.create_annihilation_operator(n_ph)
            b_dagger = self.create_creation_operator(n_ph)
            N = self.create_number_operator(n_ph)

        elif coherent_state:
            b = self.create_cs_annihilation_operator(n_ph, _z)
            b_dagger = self.create_cs_creation_operator(n_ph, _z)
            N = self.create_cs_number_operator(n_ph, _z)


        # Electronic energy contribution (diagonal in matter space, identity in photon space)
        E_matter = np.diag(self.E_array[:n_el])
        E = np.kron(I_photon, E_matter)

        # Photon energy contribution (diagonal in photon space, identity in matter space)
        Cav_photon = omega * N
        Cav = np.kron( Cav_photon, I_matter)

        #Bilinear light-matter coupling term

        BLC_matter = _d
        BLC_photon = (b_dagger + b)

        BLC = - np.sqrt(omega / 2)  *  np.kron( BLC_photon, I_matter) @ np.kron(I_photon, BLC_matter)


        # Dipole -energy term
        DSE_matter = 1 / 2 * _d @ _d

        DSE = np.kron( I_photon, DSE_matter)

        # Get the diagonals of DSE 
        #DSE_diag = np.zeros_like(DSE)
        #np.fill_diagonal(DSE_diag, np.diagonal(DSE))

        # get off diagonals of the SE
        #DSE_off_diag = DSE - DSE_diag

        # take H0 as E + Cav + DSE Diagonals
        H0 = E + Cav # + DSE_diag

        # take perturbation as BLC + DSE_off_diag
        V= BLC + DSE #DSE_off_diag



        # Diagonalize the unperturbed Hamiltonian
        E0, psi0 = np.linalg.eigh(H0)


        #psi0[:, 0] is firt wavefunction
        print(psi0.shape)

        # Initialize corrections to energy and wavefunction
        energy_corrections = np.zeros(( order+1))
        blc_corrections = np.zeros(( order+1))
        dse_corrections = np.zeros(( order+1))
        wavefunction_corrections = np.zeros((n_max, order+1))



        # Set unperturbed energies and wavefunctions
        energy_corrections[ 0] = E0[n]
        wavefunction_corrections[ :, 0] = psi0[:, n]


        n_0 = 0


        for k in range(1, order + 1):
            
            if k == 1:
                #first order energy correction
                energy_corrections[1] = np.dot(psi0[:, n].T, np.dot(V, psi0[:,n]))
                blc_corrections[1] = np.dot(psi0[:, n].T, np.dot(BLC, psi0[:,n]))
                dse_corrections[1] = np.dot(psi0[:, n].T, np.dot(DSE, psi0[:,n]))

                print(F' 1st Order Energy Correction {energy_corrections[1]}')
                print(F' 1st Order BLC Correction    {blc_corrections[1]}')
                print(F' 1st Order DSE Correction    {dse_corrections[1]}')


                coeff_m = 0
                for m in range(0, n_max):
                    coeff_m = 0
                    if m!= n:


                        coeff_m -=   np.dot(psi0[:, m].T, np.dot(V, psi0[:,n]))

                        for l in range(1, k+1):
                            #goes from l to k-1
                            coeff_m += energy_corrections[l] * np.dot(psi0[:, m].T, wavefunction_corrections[:, k-l])

                        coeff_m = coeff_m/np.abs(E0[m] - E0[n])

                        wavefunction_corrections[:, k ] += (coeff_m * psi0[:,m])


            if k!=1:  
                for m in range(0, n_max):
                    coeff_m = 0
                    if m!= n:
                        coeff_m -=  np.dot(psi0[:,m].T, np.dot(V, wavefunction_corrections[:, k-1]))
                        for l in range(1, k+1):
                            #goes from l to k-1
                            coeff_m += energy_corrections[l] * np.dot(psi0[:, m].T, wavefunction_corrections[:, k-l])

                        coeff_m = coeff_m/np.abs(E0[m] - E0[n])
                        # print("coeff_ m for state ", m , ": ", coeff_m)

                        wavefunction_corrections[:, k ] += (coeff_m * psi0[:,m])



            if k!= 1:
                energy_correction =  np.dot(psi0[:,n].T, np.dot(V, wavefunction_corrections[:, k-1]))
                blc_correction =  np.dot(psi0[:,n].T, np.dot(BLC, wavefunction_corrections[:, k-1]))
                dse_correction =  np.dot(psi0[:,n].T, np.dot(DSE, wavefunction_corrections[:, k-1]))
                
                for j in range(0, k):
                    if j != k:
                        #print("energy correction for ", j ," : " ,energy_correction)
                        energy_correction -=  energy_corrections[j]  * np.dot(psi0[:,n].T,  wavefunction_corrections[:, k-j])
                        blc_correction -=  blc_corrections[j]  * np.dot(psi0[:,n].T,  wavefunction_corrections[:, k-j])
                        dse_correction -=  dse_corrections[j]  * np.dot(psi0[:,n].T,  wavefunction_corrections[:, k-j])

                energy_corrections[ k] = energy_correction
                blc_corrections[ k] = blc_correction
                dse_corrections[ k] = dse_correction


        #print("energy_corrections: " , energy_corrections)
        return energy_corrections, blc_corrections, dse_corrections

                    

    def PQED_Hamiltonian(self, n_el, n_ph, omega, lambda_vector,  coherent_state = False):
        """
        Build the PF Hamiltonian for a system with n_el electronic states and n_ph photon states.
        """

        # Identity matrices for each subsystem
        I_matter = np.eye(n_el)
        I_photon = np.eye(n_ph)

        # Build dipole moment matrix _d
        d_array = self.build_d_array(
                n_el, lambda_vector, self.dipoles)
        _d = np.copy(d_array)

        # assign z for the coherent state transformation
        _z = -_d[0,0] / np.sqrt(2 * omega)


        # Create bosonic subspace operators
        if coherent_state == False:
            b = self.create_annihilation_operator(n_ph)
            b_dagger = self.create_creation_operator(n_ph)
            N = self.create_number_operator(n_ph)

        elif coherent_state:
            b = self.create_cs_annihilation_operator(n_ph, _z)
            b_dagger = self.create_cs_creation_operator(n_ph, _z)
            N = self.create_cs_number_operator(n_ph, _z)

        # Electronic energy contribution (diagonal in matter space, identity in photon space)
        E_matter = np.diag(self.E_array[:n_el])
        E = np.kron(I_photon, E_matter)

        # Photon energy contribution (diagonal in photon space, identity in matter space)
        Cav_photon = omega * N
        Cav = np.kron( Cav_photon, I_matter)

        #Bilinear light-matter coupling term

        BLC_matter = _d
        BLC_photon = (b_dagger + b)

        BLC = - np.sqrt(omega / 2)  *  np.kron( BLC_photon, I_matter) @ np.kron(I_photon, BLC_matter)


        # Dipole -energy term
        DSE_matter = 1 / 2 * _d @ _d

        DSE = np.kron( I_photon, DSE_matter)

        # Total Hamiltonian
        H = E + Cav  + BLC + DSE


        self.PCQED_MU = np.kron(I_photon, _d)


        # Return eigenvalues of the Hamiltonian
        eigenvalues, eigenvectors = np.linalg.eigh(H)
        return eigenvalues, eigenvectors, H



In [ ]:
# read data from .npy files for formaldehyde casci(8,8) calculations
#Drive link for .npy files of LiH

# https://drive.google.com/drive/folders/1NQvqQ4KiXTmZZ3OjCM5nA2dqIs5ete9X?usp=sharing

# !!! Change this to the correct path on your computer!
#npy_folder = "/Users/proden/Code/POLARITONPERTURBATIONTHEORY/data/"
npy_folder = "/Users/jayfoley/Code/data_repository/Mapol/LiH/PCQED/6-311G/r_scan/" 


# these file names should still be good
E_npy_file = npy_folder + "LiH_r_scan_6311g_fci_tight_davidson_Energies.npy"
Mu_npy_file = npy_folder + "LiH_r_scan_6311g_fci_tight_davidson_Dipoles.npy"

# store energy eigenvalues in E_array
E_array = np.load(E_npy_file)
# store dipole matrix elements in Mu_array
Mu_array = np.load(Mu_npy_file)

print(np.shape(E_array))



In [ ]:
N_R = 1

d_array = np.linspace(1.4, 2.2, N_R)
N_l = len(d_array)
N_el = 300
N_ph = 6
omega = 0.12086
lambda_vector = np.array([0, 0, 0.1])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pn_pcqed = np.zeros((N_l, N_el * N_ph))
_cs_pcqed = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0
for d in d_array:
    instance= PT_arbitrary(E_array[:,ctr], Mu_array[:,:,:,ctr])  # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pn_pcqed[ctr, :]  = instance.PQED_Hamiltonian(N_el, N_ph, omega, lambda_vector)[0]
    _cs_pcqed[ctr, :]  = instance.PQED_Hamiltonian(N_el, N_ph, omega, lambda_vector, coherent_state=True)[0]
    
    

    pn_pt_energies = instance.compute_energy_corrections(5, N_el, N_ph, omega, lambda_vector)
    cs_pt_energies = instance.compute_energy_corrections(5, N_el, N_ph, omega, lambda_vector, coherent_state=True)

    ctr += 1

    #print(np.sum(instance.compute_energy_corrections(9, N_el, N_ph, omega, lambda_vector)))

PN_PT_to_Full_Order = np.sum(pn_pt_energies)
CS_PT_to_Full_Order = np.sum(cs_pt_energies)

print(F'PN-PCQED {_pn_pcqed[0, 0]}')
print(F'CS-PCQED {_cs_pcqed[0, 0]}')
print(F'PN-PT-N  {PN_PT_to_Full_Order}')
print(F'CS-PT-N  {CS_PT_to_Full_Order}')

dE_PN_CS_Var = _pn_pcqed[0, 0] - _cs_pcqed[0, 0]
dE_PNPT_CS_Var = PN_PT_to_Full_Order - _cs_pcqed[0, 0]
dE_CSPT_CS_Var = CS_PT_to_Full_Order - _cs_pcqed[0, 0]

print(F'PN-PCQED is {dE_PN_CS_Var:12.10e} above CS-PCQED')
print(F'PN-PTN   is {dE_PNPT_CS_Var:12.10e} above CS-PCQED')
print(F'CS-PTN   is {dE_CSPT_CS_Var:12.10e} above CS-PCQED')


print(pn_pt_energies)
print(cs_pt_energies)

assert np.isclose(_pn_pcqed[0, 0], -8.009572189531458, 1e-9, 1e-9)
assert np.isclose(_cs_pcqed[0, 0], -8.009572189882249, 1e-9, 1e-9)


In [ ]:
print(pn_pt_energies)
print(cs_pt_energies)

In [ ]:
print(pn_blc_corrections)
print(cs_blc_corrections)

print(pn_dse_corrections)
print(cs_dse_corrections)

In [ ]:


N_R = 1

d_array = np.linspace(1.4, 2.2, N_R)
N_l = len(d_array)
N_el = 100
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0, 0, 0.05])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pcqed_50010 = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0
for d in d_array:
    instance= PT_arbitrary(E_array[:,ctr], Mu_array[:,:,:,ctr])  # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pcqed_50010[ctr, :]  = instance.PQED_Hamiltonian(N_el, N_ph, omega, lambda_vector)[0]

    print(np.sum(instance.compute_energy_corrections(9, N_el, N_ph, omega, lambda_vector, coherent_state=True, coherent_state_pos=0)))

    print(_pcqed_50010[ctr, 0])
    ctr += 1

In [ ]:


N_R = 1

d_array = np.linspace(1.4, 2.2, N_R)
N_l = len(d_array)
N_el = 200
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0, 0, 0.05])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pcqed_20010 = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0
for d in d_array:
    instance= PT_arbitrary(E_array[:,ctr], Mu_array[:,:,:,ctr])  # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pcqed_20010[ctr, :]  = instance.PQED_Hamiltonian(N_el, N_ph, omega, lambda_vector)[0]

    #print(np.sum(instance.compute_energy_corrections(9, N_el, N_ph, omega, lambda_vector, coherent_state=True, coherent_state_pos=0)))

    print(_pcqed_20010[ctr, 0])
    ctr += 1

In [ ]:

orders = [0,1,2,3,4,5,6,7,8,9,10]

pt_cs = []
pt_pn = []

N_R = 1

d_array = np.linspace(1.4, 2.2, N_R)
N_l = len(d_array)
N_el = 200
N_ph = 3
omega = 0.12086
lambda_vector = np.array([0, 0, 0.05])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pcqed_100_3_pn = np.zeros((N_l, N_el * N_ph))
_pcqed_100_3_cs = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0

for d in d_array:
    instance= PT_arbitrary(E_array[:,ctr], Mu_array[:,:,:,ctr])  # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pcqed_100_3_pn[ctr, :]  = instance.PQED_Hamiltonian(N_el, N_ph, omega, lambda_vector)[0]
    _pcqed_100_3_cs[ctr, :]  = instance.PQED_Hamiltonian(N_el, N_ph, omega, lambda_vector, coherent_state=True,  coherent_state_pos=0)[0]

    for order in orders:
        pt_pn.append(np.sum(instance.compute_energy_corrections(order, N_el, N_ph, omega, lambda_vector)))
        pt_cs.append(np.sum(instance.compute_energy_corrections(order, N_el, N_ph, omega, lambda_vector, coherent_state=True, coherent_state_pos=0)))

    print(_pcqed_50010[ctr, 0])
    ctr += 1

In [ ]:
plt.axhline(y = _pcqed_100_3_pn[0, 0], label = "PN" , color = "green")
plt.axhline(y = _pcqed_100_3_cs[0, 0], label = "CS", color = "blue")

plt.plot(pt_pn, color = "green")
plt.plot(pt_cs, color = "blue")

plt.ylabel("energy (Hartree)")
plt.xlabel("pertrubation order")
plt.legend()


plt.ylim(-8.0097, -8.0096)

In [ ]:

plt.plot(np.abs(pt_pn - _pcqed_20010[0, 0]), color = "green")
plt.plot(np.abs(pt_cs- _pcqed_20010[0, 0]), color = "blue")

plt.ylabel("energy (Hartree)")
plt.xlabel("pertrubation order")
plt.legend()

plt.ylim(0,0.0001)


